<a href="https://colab.research.google.com/github/Nastiiasaenko/Final-Project---Explainable-AI-/blob/main/notebooks/Global_Dataset_Enrichment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np

In [4]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Specify the path to your desired folder in Drive
drive_folder = '/content/drive/MyDrive/MIDS/XAI/XAI_Final_project'

# Ensure the folder exists
os.makedirs(drive_folder, exist_ok=True)

# Change the current working directory to the specified folder
os.chdir(drive_folder)

# Confirm the working directory
print("Current working directory:", os.getcwd())



Mounted at /content/drive
Current working directory: /content/drive/MyDrive/MIDS/XAI/XAI_Final_project


In [5]:
global_df = pd.read_csv('global_dataset.csv')

<ipython-input-5-07264a2067b1>:1: DtypeWarning: Columns (5,6,8,10,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  global_df = pd.read_csv('global_dataset.csv')


In [10]:
global_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240156 entries, 0 to 240155
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   title               120219 non-null  object 
 1   body                202040 non-null  object 
 2   source              240150 non-null  object 
 3   timestamp           136998 non-null  object 
 4   Misinfo_flag        16588 non-null   float64
 5   type_of_misinfo     3193 non-null    object 
 6   type_reddit         36336 non-null   object 
 7   topic               210148 non-null  object 
 8   subtopic            42396 non-null   object 
 9   entities            239180 non-null  object 
 10  Polarization_flag   75090 non-null   object 
 11  	Misinfo_flag       1627 non-null    float64
 12  type_of_content     25654 non-null   object 
 13  potential_prompt0   51566 non-null   object 
 14  hashtags            6329 non-null    object 
 15  gender              41104 non-null

## Predicting sentiment

In [38]:
sentiment_bodies = pd.Series(np.where(global_df['body'].isnull(), global_df['title'], global_df['body']))

In [28]:
sentiment_bodies.head()

,0
0,Health Canada approves AstraZeneca COVID-19 va...
1,COVID-19 in Canada: 'Vaccination passports' a ...
2,Coronavirus variants could fuel Canada's third...
3,Canadian government to extend COVID-19 emergen...
4,Canada: Pfizer is 'extremely committed' to mee...


In [39]:
import pandas as pd
from transformers import pipeline, AutoTokenizer
import torch
from tqdm import tqdm

# Check if GPU is available and set device accordingly
device = 0 if torch.cuda.is_available() else -1

# Load the sentiment analysis pipeline and tokenizer with GPU usage if available
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, device=device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example pandas Series with your texts
sentiment_bodies

# Ensure all elements in sentiment_bodies are strings and handle missing values
sentiment_bodies = sentiment_bodies.fillna('').astype(str)

# Define a function that splits long texts into manageable chunks
def split_texts_into_chunks(texts, max_token_length=512):
    split_texts = []

    for text in tqdm(texts, desc="Splitting texts into chunks"):
        try:
            # Tokenize the text
            tokens = tokenizer(text, return_tensors='pt', truncation=False)["input_ids"][0]

            # Split tokens into chunks of max_token_length
            chunks = [tokens[i:i + max_token_length] for i in range(0, len(tokens), max_token_length)]

            # Convert each chunk back to text
            text_chunks = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]
            split_texts.extend(text_chunks)
        except Exception as e:
            print(f"Error processing text: {text[:50]}... Error: {e}")
            continue

    return split_texts

# Split the original texts into chunks that can be processed without exceeding max length
split_texts = split_texts_into_chunks(sentiment_bodies)

# Perform batch sentiment analysis on the split texts
print("Performing batch sentiment analysis...")
batch_sentiments = list(tqdm(sentiment_pipeline(split_texts, truncation=True), desc="Sentiment analysis"))

# Aggregate results back to the original text level by majority voting for each chunk group
def aggregate_sentiments(original_texts, split_texts, batch_sentiments):
    all_results = []
    idx = 0

    for text in tqdm(original_texts, desc="Aggregating sentiments"):
        try:
            # Tokenize the text to determine how many chunks it has
            tokens = tokenizer(text, return_tensors='pt', truncation=False)["input_ids"][0]
            num_chunks = (len(tokens) + 511) // 512  # Calculate how many chunks the text was split into

            # Get the sentiment results for all chunks of this text
            sentiments = [batch_sentiments[idx + i]['label'] for i in range(num_chunks)]
            idx += num_chunks

            # Determine the overall sentiment using majority vote
            overall_sentiment = max(set(sentiments), key=sentiments.count)
            all_results.append(overall_sentiment)
        except Exception as e:
            print(f"Error processing aggregation for text: {text[:50]}... Error: {e}")
            all_results.append("UNKNOWN")

    return pd.Series(all_results)

# Aggregate the batch sentiment results back to the original texts
sentiment_series = aggregate_sentiments(sentiment_bodies, split_texts, batch_sentiments)

# Print the resulting Series
print(sentiment_series)


Splitting texts into chunks: 100%|██████████| 240156/240156 [03:18<00:00, 1209.49it/s]


Performing batch sentiment analysis...


Aggregating sentiments: 100%|██████████| 240156/240156 [02:23<00:00, 1668.78it/s]

0         POSITIVE
1         NEGATIVE
2         NEGATIVE
3         POSITIVE
4         POSITIVE
            ...   
240151    NEGATIVE
240152    NEGATIVE
240153    NEGATIVE
240154    NEGATIVE
240155    POSITIVE
Length: 240156, dtype: object


In [40]:
sentiment_series

,0
0,POSITIVE
1,NEGATIVE
2,NEGATIVE
3,POSITIVE
4,POSITIVE
...,...
240151,NEGATIVE
240152,NEGATIVE
240153,NEGATIVE
240154,NEGATIVE


In [14]:
sentiment_bodies

,0
0,Health Canada approves AstraZeneca COVID-19 va...
1,COVID-19 in Canada: 'Vaccination passports' a ...
2,Coronavirus variants could fuel Canada's third...
3,Canadian government to extend COVID-19 emergen...
4,Canada: Pfizer is 'extremely committed' to mee...
...,...
240151,Whether through callousness in targeting or si...
240152,The energy industry has become the stock marke...
240153,A Russian missile strike on a city in central ...
240154,"Konstantin Malofeev, an influential oligarch i..."


In [41]:
global_df['sentiment_generated'] = sentiment_series

In [43]:
global_df.to_csv('global_dataset_enriched.csv', index=False)

## Political stance?

In [46]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

text = "The government's new environmental policy is a welcome change, focusing on renewable energy and reducing emissions."
candidate_labels = ["left-leaning", "right-leaning", "centrist"]
result = classifier(text, candidate_labels)
print(result)


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': "The government's new environmental policy is a welcome change, focusing on renewable energy and reducing emissions.", 'labels': ['centrist', 'right-leaning', 'left-leaning'], 'scores': [0.5308507084846497, 0.3266083598136902, 0.14254093170166016]}


In [48]:
from transformers import pipeline

# Load a valid model from Hugging Face
model_name = "facebook/bart-large-mnli"
classifier = pipeline("zero-shot-classification", model=model_name)

# Example usage
text = "The government's new environmental policy is a welcome change, focusing on renewable energy and reducing emissions."
candidate_labels = ["left-leaning", "right-leaning", "centrist"]

result = classifier(text, candidate_labels)
print(result)



Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': "The government's new environmental policy is a welcome change, focusing on renewable energy and reducing emissions.", 'labels': ['centrist', 'right-leaning', 'left-leaning'], 'scores': [0.5308507084846497, 0.3266083598136902, 0.14254093170166016]}


In [49]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import torch

# Check if GPU is available and set device accordingly
device = 0 if torch.cuda.is_available() else -1

# Load the zero-shot classification model for political leaning
model_name_political = "facebook/bart-large-mnli"
classifier_political = pipeline("zero-shot-classification", model=model_name_political, device=device)

# Example pandas Series with your texts


# Define political leaning labels
candidate_labels = ["left-leaning", "right-leaning", "centrist"]

# Perform political leaning classification by selecting the highest scoring label
def classify_political_leaning(texts):
    political_results = []

    for text in tqdm(texts, desc="Classifying political leaning"):
        try:
            # Perform zero-shot classification on the text
            result = classifier_political(text, candidate_labels)

            # Extract the label with the highest score
            highest_score_idx = result['scores'].index(max(result['scores']))
            predicted_label = result['labels'][highest_score_idx]

            political_results.append(predicted_label)
        except Exception as e:
            print(f"Error processing text for political leaning: {text[:50]}... Error: {e}")
            political_results.append("UNKNOWN")

    return pd.Series(political_results)

# Classify the political leaning of the original texts
political_series = classify_political_leaning(sentiment_bodies)

# Print the resulting Series for political leaning classification
print("Political Leaning Series:")
print(political_series)


Classifying political leaning:   5%|▌         | 13160/240156 [13:34<3:54:03, 16.16it/s]


KeyboardInterrupt: 